In [16]:
import re
from tqdm import tqdm_notebook
from nltk.corpus import stopwords
from tensorflow.keras import regularizers, initializers, optimizers, callbacks
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np

import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

In [31]:
df = pd.read_csv("train2.tsv", sep='\t')
df.columns = ['drop', 'ID', 'label', 'statement', 'subject', 'speaker', 'speaker title', 'state', 'party', 'barely_count', 'false_count', 'half_count', 'mostly_count', 'fire_count', 'location', 'context']
del df['drop']
df.head()

def int_decode(label):
    if label == 'true':
        label=0
    elif label == 'half-true':
        label=0
    elif label == 'barely-true':
        label=1
    elif label == 'mostly-true':
        label=0
    elif label == 'false':
        label=1
    elif label == 'pants-fire':
        label=1
    return label

label_new = []
label_old = df['label']
for i in range(0, len(df)):
	label_new.append(int_decode(label_old[i]))

x = df["statement"]
y = label_new
y = np.array(y)

MAX_NB_WORDS = 100000 # max number of words for tokenizer
MAX_SEQUENCE_LENGTH = 1000 # max length of each sentences, including padding
VALIDATION_SPLIT = 0.2 # 20% of data for validation (not used in training)
EMBEDDING_DIM = 100 # embedding dimensions for word vectors
GLOVE_DIR = "glove6B/glove.6B."+str(EMBEDDING_DIM)+"d.txt"

def clean_text(text, remove_stopwords = True):
    output = ""
    text = str(text).replace(r'http[\w:/\.]+','') # removing urls
    text = str(text).replace(r'[^\.\w\s]','') #remove everything but characters and punctuation
    text = str(text).replace(r'\.\.+','.') #replace multiple periods with a single one
    text = str(text).replace(r'\.',' . ') #replace periods with a single one
    text = str(text).replace(r'\s\s+',' ') #replace multiple white space with a single one
    text = str(text).replace('\n', '') #removing line breaks
    text = re.sub(r'[^\w\s]','',text).lower() #lower texts
    if remove_stopwords:
        text = text.split(" ")
        for word in text:
            if word not in stopwords.words("english"):
                output+=word
            else:
                output=output
    return str(output.strip())[1:-3].replace(" ", " ")






(10239,)


AttributeError: 'Series' object has no attribute 'int_encode'

In [18]:
import nltk
nltk.download('stopwords')

texts = [] 

for line in x:
    texts.append(clean_text(line))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
print(texts[0])

declinecoalstartstartednaturalgastookstartedbeginpresidentgeorgewbushsadministration


In [20]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print('Vocabulary size:', len(word_index))

Vocabulary size: 10405


In [21]:
data = pad_sequences(sequences, padding = 'post', maxlen = MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', y.shape)

Shape of data tensor: (10239, 1000)
Shape of label tensor: (10239,)


In [22]:
indices = np.arange(data.shape[0])
data = data[indices]
labels = y[indices]


num_validation_samples = int(VALIDATION_SPLIT*data.shape[0])
x_train = data[: -num_validation_samples]
y_train = labels[: -num_validation_samples]
x_val = data[-num_validation_samples: ]
y_val = labels[-num_validation_samples: ]
print('Number of entries in each category:')
print('training: ', y_train.sum(axis=0))
print('validation: ', y_val.sum(axis=0))

Number of entries in each category:
training:  3562
validation:  925


In [23]:
embeddings_index = {}
f = open(GLOVE_DIR, encoding='utf8')
print('Loading Glove from:', GLOVE_DIR,'…', end='')
for line in f:
    values = line.split()
    word = values[0]
    embeddings_index[word] = np.asarray(values[1:], dtype='float32')
f.close()
print('Done.\n Proceeding with Embedding Matrix…', end='')
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print('Completed!')

Loading Glove from: glove6B/glove.6B.100d.txt …Done.
 Proceeding with Embedding Matrix…Completed!


In [24]:
model = Sequential()
model.add(Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32'))
model.add(Embedding(len(word_index) + 1,
 EMBEDDING_DIM,
 weights = [embedding_matrix],
 input_length = MAX_SEQUENCE_LENGTH,
 trainable=False,
 name = 'embeddings'))
model.add(LSTM(60, return_sequences=True,name='lstm_layer'))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.1))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(6, activation='sigmoid'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [25]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [26]:
history = model.fit(x_train, y_train, epochs = 10, batch_size=128, validation_data=(x_val, y_val))

ValueError: A target array with shape (8192, 1) was passed for an output of shape (None, 6) while using as loss `binary_crossentropy`. This loss expects targets to have the same shape as the output.